# Public FInal

In [ ]:
import pandas as pd
import numpy as np
# Function to find IDs to replace values less than or equal to 10 with the average of other values for 21 days
# Now considers minimum value as 1 if it's zero
def find_ids_to_replace_10x_v2(df):
    ids_to_replace = []
    prediction_cols = df.columns[1:22]  # Assuming the first column is 'ID' and the next 21 columns are the prediction days

    for _, row in df.iterrows():
        values = row[prediction_cols].values
        min_val = np.min(values)
        max_val = np.max(values)

        min_val = max(min_val, 1)  # If minimum value is 0, consider it as 1

        if max_val / min_val >= 10:  # To avoid division by zero, replace min_val with 1 if it's zero
            if any(val <= 10 for val in values):
                ids_to_replace.append(row['ID'])

    return ids_to_replace

# Function to replace values less than or equal to 10 with the average of other values for those IDs
# Improved to handle edge cases like negative infinity
def replace_10x_with_avg_improved(df, ids_to_replace):
    prediction_cols = df.columns[1:22]  # Assuming the first column is 'ID' and the next 21 columns are the prediction days
    df_replaced = df.copy()

    for idx in ids_to_replace:
        specific_row = df_replaced[df_replaced['ID'] == idx]
        row_values = specific_row[prediction_cols].values[0]
        avg_value = np.mean([val for val in row_values if val > 10])

        # Handle edge cases where all values are <= 10 (preventing negative infinity)
        if np.isnan(avg_value) or np.isinf(avg_value):
            continue

        row_values[row_values <= 10] = avg_value
        df_replaced.loc[df_replaced['ID'] == idx, prediction_cols] = row_values

    return df_replaced

# Initialize a dictionary to store the new DataFrames
dfs_replaced_10x_improved = {}

# Loop through each file to apply the changes
file_paths_improved = [
    '/content/lstm_35.csv',
    '/content/DLinear_0825_724.csv',
    '/content/lstf_jh_777.csv'
]

for file_path in file_paths_improved:
    df = pd.read_csv(file_path)

    # Find IDs that need to have values less than or equal to 10 replaced
    ids_to_replace_10x_v2 = find_ids_to_replace_10x_v2(df)

    # Replace the values with the improved function
    df_replaced_10x_improved = replace_10x_with_avg_improved(df, ids_to_replace_10x_v2)

    # Save the new DataFrame to a CSV file
    new_file_path_10x_improved = file_path.replace('.csv', '_pp.csv')
    df_replaced_10x_improved.to_csv(new_file_path_10x_improved, index=False)

    dfs_replaced_10x_improved[new_file_path_10x_improved] = df_replaced_10x_improved

dfs_replaced_10x_improved.keys()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


dict_keys(['/content/lstm_35_pp.csv', '/content/DLinear_0825_724_pp.csv', '/content/lstf_jh_777_pp.csv'])

In [ ]:
import pandas as pd
import numpy as np
# Reload the provided files for ensemble
lstm = pd.read_csv('/content/lstm_35_pp.csv')
DL = pd.read_csv('/content/DLinear_0825_724_pp.csv')
LL = pd.read_csv('/content/lstf_jh_777_pp.csv')


ensemble_6files_df = DL[['ID']].copy()

weights = {
    'lstm': 0.4,
    'DLinear' : 0.3,
    'LLinear' : 0.3,
}


for date_column in DL.columns[1:]:
    ensemble_6files_df[date_column] = (
        lstm[date_column] * weights['lstm'] +
        DL[date_column] * weights['DLinear'] +
        LL[date_column] * weights['LLinear']
    )


ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)
import pandas as pd
import numpy as np
train_data = pd.read_csv("/content/drive/MyDrive/Aimers3/train.csv")

# Filter dates from 2022-01-01 to 2023-04-04 for train ensemble_6files_df
date_cols = [col for col in train_data.columns if '2022-01-01' <= col <= '2023-04-04']
filtered_train_data = train_data[['ID'] + date_cols]

# Filter the IDs from bi3gr3lstf2_fl.csv where at least one of the 21 days values is 0
ids_at_least_one_zero_fl = ensemble_6files_df[ensemble_6files_df.iloc[:, 1:].apply(lambda x: (x == 0).any(), axis=1)]['ID']

# Calculate the median for the filtered IDs in train.csv, excluding 0 values
medians_fl_at_least_one_zero = filtered_train_data[filtered_train_data['ID'].isin(ids_at_least_one_zero_fl)].iloc[:, 1:].apply(lambda x: x[x != 0].median() if (x[x != 0].shape[0] > 0) else 0, axis=1)

# Replace the values in bi3gr3lstf2_fl.csv with the calculated medians
for idx in ids_at_least_one_zero_fl:
    ensemble_6files_df.loc[ensemble_6files_df['ID'] == idx, ensemble_6files_df.columns[1:]] = medians_fl_at_least_one_zero[idx]

ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].replace(0, 1)

means = ensemble_6files_df.iloc[:, 1:].mean(axis=1)

for column in ensemble_6files_df.columns[1:]:
    ensemble_6files_df[column] = means
ensemble_6files_df = ensemble_6files_df.round().astype(int)
data = ensemble_6files_df.copy()
data.iloc[:, 1:] = data.iloc[:, 1:].replace(0, 1)
means = data.iloc[:, 1:].mean(axis=1)

# Replace the 21 days values with their mean for each ID
for column in data.columns[1:]:
    data[column] = means
data = data.round().astype(int)
data.to_csv('re.csv',index=False)
data

<ipython-input-15-f929c19247c5>:26: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)


,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,4,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
15886,15886,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
15887,15887,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
15888,15888,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


# Real_Final

In [ ]:
import pandas as pd
import numpy as np
# Function to find IDs to replace values less than or equal to 10 with the average of other values for 21 days
# Now considers minimum value as 1 if it's zero
def find_ids_to_replace_10x_v2(df):
    ids_to_replace = []
    prediction_cols = df.columns[1:22]  # Assuming the first column is 'ID' and the next 21 columns are the prediction days

    for _, row in df.iterrows():
        values = row[prediction_cols].values
        min_val = np.min(values)
        max_val = np.max(values)

        min_val = max(min_val, 1)  # If minimum value is 0, consider it as 1

        if max_val / min_val >= 10:  # To avoid division by zero, replace min_val with 1 if it's zero
            if any(val <= 10 for val in values):
                ids_to_replace.append(row['ID'])

    return ids_to_replace

# Function to replace values less than or equal to 10 with the average of other values for those IDs
# Improved to handle edge cases like negative infinity
def replace_10x_with_avg_improved(df, ids_to_replace):
    prediction_cols = df.columns[1:22]  # Assuming the first column is 'ID' and the next 21 columns are the prediction days
    df_replaced = df.copy()

    for idx in ids_to_replace:
        specific_row = df_replaced[df_replaced['ID'] == idx]
        row_values = specific_row[prediction_cols].values[0]
        avg_value = np.mean([val for val in row_values if val > 10])

        # Handle edge cases where all values are <= 10 (preventing negative infinity)
        if np.isnan(avg_value) or np.isinf(avg_value):
            continue

        row_values[row_values <= 10] = avg_value
        df_replaced.loc[df_replaced['ID'] == idx, prediction_cols] = row_values

    return df_replaced

# Initialize a dictionary to store the new DataFrames
dfs_replaced_10x_improved = {}

# Loop through each file to apply the changes
file_paths_improved = [
    #'/content/DLinear_0827_41.csv',
    #'/content/DLinear_0827_42.csv',
    #'/content/DLinear_0827_43.csv',
    #'/content/DLinear_0827_44.csv',
    #'/content/DLinear_0826_19990313.csv',
    #'/content/DLinear_0826_990313.csv',
    #'/content/lstf_jh_41.csv',
    #'/content/lstf_jh_42.csv',
    #'/content/lstf_jh_43.csv',
    #'/content/lstf_jh_44.csv',
    #'/content/lstf_jh_45.csv',
    '/content/lstm_35.csv',
    #'/content/lstm_35_43_jb.csv',
    #'/content/lstm_28_43_jb.csv'
    '/content/DLinear_0825_724.csv',
    '/content/lstf_jh_777.csv'
]

for file_path in file_paths_improved:
    df = pd.read_csv(file_path)

    # Find IDs that need to have values less than or equal to 10 replaced
    ids_to_replace_10x_v2 = find_ids_to_replace_10x_v2(df)

    # Replace the values with the improved function
    df_replaced_10x_improved = replace_10x_with_avg_improved(df, ids_to_replace_10x_v2)

    # Save the new DataFrame to a CSV file
    new_file_path_10x_improved = file_path.replace('.csv', '_pp.csv')
    df_replaced_10x_improved.to_csv(new_file_path_10x_improved, index=False)

    dfs_replaced_10x_improved[new_file_path_10x_improved] = df_replaced_10x_improved

dfs_replaced_10x_improved.keys()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


dict_keys(['/content/lstm_35_pp.csv', '/content/DLinear_0825_724_pp.csv', '/content/lstf_jh_777_pp.csv'])

In [ ]:
import pandas as pd
import numpy as np
# Reload the provided files for ensemble
DLinear_41 = pd.read_csv('/content/DLinear_0827_41_pp.csv')
DLinear_42 = pd.read_csv('/content/DLinear_0827_42_pp.csv')
DLinear_43 = pd.read_csv('/content/DLinear_0827_43_pp.csv')
DLinear_44 = pd.read_csv('/content/DLinear_0827_44_pp.csv')
DLinear_990313 = pd.read_csv('/content/DLinear_0826_990313_pp.csv')
DLinear_19990313 = pd.read_csv('/content/DLinear_0826_19990313_pp.csv')
Linear_41 = pd.read_csv('/content/lstf_jh_41_pp.csv')
Linear_42 = pd.read_csv('/content/lstf_jh_42_pp.csv')
Linear_43 = pd.read_csv('/content/lstf_jh_43_pp.csv')
Linear_44 = pd.read_csv('/content/lstf_jh_44_pp.csv')
Linear_45 = pd.read_csv('/content/lstf_jh_45_pp.csv')
lstm_28 = pd.read_csv('/content/lstm_28_43_jb_pp.csv')
lstm_35_43 = pd.read_csv('/content/lstm_35_43_jb_pp.csv')
lstm_35_42 = pd.read_csv('/content/lstm_35_pp.csv')


#DLinear = pd.read_csv('/content/DLinear_0825_replaced_10x_improved (1).csv')
#lstm105_df = pd.read_csv('/content/bilstm_nofeature_105_imp3_noiseX_41.csv')
#lstm35_df = pd.read_csv('/content/bilstm_nofeature_35_imp3_noiseX_41 (1).csv')
#lstm70_df = pd.read_csv('/content/bilstm_nofeature_70_imp3_noiseX_41.csv')
#gru105_df = pd.read_csv('/content/gru_nofeature_105_imp3_noiseX_41.csv')
#gru35_df = pd.read_csv('/content/gru_nofeature_35_imp3_noiseX_41.csv')
#gru70_df = pd.read_csv('/content/gru_nofeature_70_imp3_noiseX_41.csv')
#Linear = pd.read_csv('/content/lstf_jh_777_replaced_10x_improved (1).csv')
#NLinear35_df = pd.read_csv('/content/NLinear_35_.csv')
#lstm = pd.read_csv('/content/lstm_35_replaced_10x_improved (1).csv')
#lstm35 = pd.read_csv('/content/new_ensemble_result_35.csv')
#lstm56 = pd.read_csv('/content/seventh_ensemble_result_56.csv')
#lstm70 = pd.read_csv('/content/lstm_7041.csv')


ensemble_6files_df = DLinear_41[['ID']].copy()

weights = {
    'DLinear41': 0.05,
    'DLinear42' : 0.05,
    'DLinear43' : 0.05,
    'DLinear44' : 0.05,
    'DLinear990313' : 0.05,
    'DLinear19990313' : 0.05,
    'Linear41' : 0.06,
    'Linear42' : 0.06,
    'Linear43' : 0.06,
    'Linear44' : 0.06,
    'Linear45' : 0.06,
    'lstm28' : 0.12,
    'lstm35_42' : 0.16,
    'lstm35_43' : 0.12
}


for date_column in DLinear_41.columns[1:]:
    ensemble_6files_df[date_column] = (
        DLinear_41[date_column] * weights['DLinear41'] +
        DLinear_42[date_column] * weights['DLinear42'] +
        DLinear_43[date_column] * weights['DLinear43'] +
        DLinear_44[date_column] * weights['DLinear44'] +
        DLinear_990313[date_column] * weights['DLinear990313'] +
        DLinear_19990313[date_column] * weights['DLinear19990313'] +
        Linear_41[date_column] * weights['Linear41'] +
        Linear_42[date_column] * weights['Linear42'] +
        Linear_43[date_column] * weights['Linear43'] +
        Linear_44[date_column] * weights['Linear44'] +
        Linear_45[date_column] * weights['Linear45'] +
        lstm_28[date_column] * weights['lstm28'] +
        lstm_35_43[date_column] * weights['lstm35_43'] +
        lstm_35_42[date_column] * weights['lstm35_42']
    )


ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)
import pandas as pd
import numpy as np
train_data = pd.read_csv("/content/drive/MyDrive/Aimers3/train.csv")

# Filter dates from 2022-01-01 to 2023-04-04 for train ensemble_6files_df
date_cols = [col for col in train_data.columns if '2022-01-01' <= col <= '2023-04-04']
filtered_train_data = train_data[['ID'] + date_cols]

# Filter the IDs from bi3gr3lstf2_fl.csv where at least one of the 21 days values is 0
ids_at_least_one_zero_fl = ensemble_6files_df[ensemble_6files_df.iloc[:, 1:].apply(lambda x: (x == 0).any(), axis=1)]['ID']

# Calculate the median for the filtered IDs in train.csv, excluding 0 values
medians_fl_at_least_one_zero = filtered_train_data[filtered_train_data['ID'].isin(ids_at_least_one_zero_fl)].iloc[:, 1:].apply(lambda x: x[x != 0].median() if (x[x != 0].shape[0] > 0) else 0, axis=1)

# Replace the values in bi3gr3lstf2_fl.csv with the calculated medians
for idx in ids_at_least_one_zero_fl:
    ensemble_6files_df.loc[ensemble_6files_df['ID'] == idx, ensemble_6files_df.columns[1:]] = medians_fl_at_least_one_zero[idx]

ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].replace(0, 1)

means = ensemble_6files_df.iloc[:, 1:].mean(axis=1)

for column in ensemble_6files_df.columns[1:]:
    ensemble_6files_df[column] = means
ensemble_6files_df = ensemble_6files_df.round().astype(int)
data = ensemble_6files_df.copy()
data.iloc[:, 1:] = data.iloc[:, 1:].replace(0, 1)
means = data.iloc[:, 1:].mean(axis=1)

# Replace the 21 days values with their mean for each ID
for column in data.columns[1:]:
    data[column] = means
data = data.round().astype(int)
data.to_csv('Plz.csv',index=False)
data

# test

In [ ]:
import pandas as pd
import numpy as np
# Reload the provided files for ensemble
DLinear_41 = pd.read_csv('/content/lstm_35_pp.csv')
DLinear_42 = pd.read_csv('/content/lstf_jh_777_pp.csv')
DLinear_43 = pd.read_csv('/content/DLinear_0825_724_pp.csv')




# 'ID'를 기준으로 세 개의 DataFrame을 하나로 합친다.
combined_df = pd.concat([DLinear_41, DLinear_42.drop('ID', axis=1), DLinear_43.drop('ID', axis=1)], axis=1, keys=['DLinear_41', 'DLinear_42', 'DLinear_43'])

# 결과를 저장할 DataFrame을 생성한다.
ensemble_6files_df = DLinear_41[['ID']].copy()

# 각 날짜별로 중앙값을 계산한다.
for date_column in DLinear_41.columns[1:]:
    ensemble_6files_df[date_column] = combined_df.xs(date_column, axis=1, level=1).apply(lambda row: row.median(), axis=1)

ensemble_6files_df

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0
3,3,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0
4,4,4.0,4.0,4.0,4.0,5.0,5.0,4.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,10.0,13.0,15.0,16.0,16.0,15.0,15.0,15.0,16.0,...,17.0,17.0,16.0,16.0,16.0,16.0,17.0,17.0,17.0,16.0
15886,15886,5.0,6.0,7.0,8.0,8.0,8.0,8.0,8.0,7.0,...,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
15887,15887,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
15888,15888,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [ ]:
import pandas as pd
import numpy as np
# Reload the provided files for ensemble
DLinear_41 = pd.read_csv('/content/lstm_35_pp.csv')
DLinear_42 = pd.read_csv('/content/lstf_jh_777_pp.csv')
DLinear_43 = pd.read_csv('/content/DLinear_0825_724_pp.csv')




# 'ID'를 기준으로 세 개의 DataFrame을 하나로 합친다.
combined_df = pd.concat([DLinear_41, DLinear_42.drop('ID', axis=1), DLinear_43.drop('ID', axis=1)], axis=1, keys=['DLinear_41', 'DLinear_42', 'DLinear_43'])

# 결과를 저장할 DataFrame을 생성한다.
ensemble_6files_df = DLinear_41[['ID']].copy()

# 각 날짜별로 중앙값을 계산한다.
for date_column in DLinear_41.columns[1:]:
    ensemble_6files_df[date_column] = combined_df.xs(date_column, axis=1, level=1).apply(lambda row: row.median(), axis=1)



#ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)
import pandas as pd
import numpy as np
train_data = pd.read_csv("/content/drive/MyDrive/Aimers3/train.csv")

# Filter dates from 2022-01-01 to 2023-04-04 for train ensemble_6files_df
date_cols = [col for col in train_data.columns if '2022-01-01' <= col <= '2023-04-04']
filtered_train_data = train_data[['ID'] + date_cols]

# Filter the IDs from bi3gr3lstf2_fl.csv where at least one of the 21 days values is 0
ids_at_least_one_zero_fl = ensemble_6files_df[ensemble_6files_df.iloc[:, 1:].apply(lambda x: (x == 0).any(), axis=1)]['ID']

# Calculate the median for the filtered IDs in train.csv, excluding 0 values
medians_fl_at_least_one_zero = filtered_train_data[filtered_train_data['ID'].isin(ids_at_least_one_zero_fl)].iloc[:, 1:].apply(lambda x: x[x != 0].median() if (x[x != 0].shape[0] > 0) else 0, axis=1)

# Replace the values in bi3gr3lstf2_fl.csv with the calculated medians
for idx in ids_at_least_one_zero_fl:
    ensemble_6files_df.loc[ensemble_6files_df['ID'] == idx, ensemble_6files_df.columns[1:]] = medians_fl_at_least_one_zero[idx]

ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].replace(0, 1)

means = ensemble_6files_df.iloc[:, 1:].mean(axis=1)

for column in ensemble_6files_df.columns[1:]:
    ensemble_6files_df[column] = means
ensemble_6files_df = ensemble_6files_df.round().astype(int)
data = ensemble_6files_df.copy()
data.iloc[:, 1:] = data.iloc[:, 1:].replace(0, 1)
means = data.iloc[:, 1:].mean(axis=1)

# Replace the 21 days values with their mean for each ID
for column in data.columns[1:]:
    data[column] = means
data = data.round().astype(int)
data.to_csv('FINAL.csv',index=False)
data

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
1,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
4,4,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
15886,15886,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
15887,15887,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
15888,15888,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


# Final Ens

In [ ]:
import pandas as pd
import numpy as np
# Reload the provided files for ensemble
DLinear35_df = pd.read_csv('/content/sixth_ensemble_result_DLinear.csv')
#lstm105_df = pd.read_csv('/content/bilstm_nofeature_105_imp3_noiseX_41.csv')
#lstm35_df = pd.read_csv('/content/bilstm_nofeature_35_imp3_noiseX_41 (1).csv')
#lstm70_df = pd.read_csv('/content/bilstm_nofeature_70_imp3_noiseX_41.csv')
#gru105_df = pd.read_csv('/content/gru_nofeature_105_imp3_noiseX_41.csv')
#gru35_df = pd.read_csv('/content/gru_nofeature_35_imp3_noiseX_41.csv')
#gru70_df = pd.read_csv('/content/gru_nofeature_70_imp3_noiseX_41.csv')
Linear35_df = pd.read_csv('/content/fifth_ensemble_result_linear.csv')
NLinear35_df = pd.read_csv('/content/NLinear_35_.csv')
lstm28 = pd.read_csv('/content/ensemble_result_28.csv')
lstm35 = pd.read_csv('/content/new_ensemble_result_35.csv')
lstm56 = pd.read_csv('/content/seventh_ensemble_result_56.csv')
#lstm70 = pd.read_csv('/content/lstm_7041.csv')


ensemble_6files_df = DLinear35_df[['ID']].copy()

weights = {
    'lstm28' : 0.15,
    'lstm35' : 0.2,
    'lstm56' : 0.15,
    #'lstm70' : 0.2,
    #'bilstm35': 0.04,
    #'bilstm70': 0.03,
    #'bilstm105': 0.03,
    #'gru35': 0.04,
    #'gru70': 0.03,
    #'gru105': 0.03,
    'NLinear35' : 0.1,
    'DLinear35': 0.2,
    'Linear35': 0.2
}


for date_column in DLinear35_df.columns[1:]:
    ensemble_6files_df[date_column] = (
        #lstm35_df[date_column] * weights['bilstm35'] +
        #lstm70_df[date_column] * weights['bilstm70'] +
        #lstm105_df[date_column] * weights['bilstm105'] +
        #gru35_df[date_column] * weights['gru35'] +
        #gru70_df[date_column] * weights['gru70'] +
        #gru105_df[date_column] * weights['gru105'] +
        DLinear35_df[date_column] * weights['DLinear35'] +
        Linear35_df[date_column] * weights['Linear35'] +
        lstm35[date_column] * weights['lstm35'] +
        lstm28[date_column] * weights['lstm28'] +
        lstm56[date_column] * weights['lstm56'] +
        NLinear35_df[date_column] * weights['NLinear35']
    )


ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)
import pandas as pd
import numpy as np
train_data = pd.read_csv("/content/drive/MyDrive/Aimers3/train.csv")

# Filter dates from 2022-01-01 to 2023-04-04 for train ensemble_6files_df
date_cols = [col for col in train_data.columns if '2022-01-01' <= col <= '2023-04-04']
filtered_train_data = train_data[['ID'] + date_cols]

# Filter the IDs from bi3gr3lstf2_fl.csv where at least one of the 21 days values is 0
ids_at_least_one_zero_fl = ensemble_6files_df[ensemble_6files_df.iloc[:, 1:].apply(lambda x: (x == 0).any(), axis=1)]['ID']

# Calculate the median for the filtered IDs in train.csv, excluding 0 values
medians_fl_at_least_one_zero = filtered_train_data[filtered_train_data['ID'].isin(ids_at_least_one_zero_fl)].iloc[:, 1:].apply(lambda x: x[x != 0].median() if (x[x != 0].shape[0] > 0) else 0, axis=1)

# Replace the values in bi3gr3lstf2_fl.csv with the calculated medians
for idx in ids_at_least_one_zero_fl:
    ensemble_6files_df.loc[ensemble_6files_df['ID'] == idx, ensemble_6files_df.columns[1:]] = medians_fl_at_least_one_zero[idx]

ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].replace(0, 1)

means = ensemble_6files_df.iloc[:, 1:].mean(axis=1)

for column in ensemble_6files_df.columns[1:]:
    ensemble_6files_df[column] = means
ensemble_6files_df = ensemble_6files_df.round().astype(int)
data = ensemble_6files_df.copy()
data.iloc[:, 1:] = data.iloc[:, 1:].replace(0, 1)
means = data.iloc[:, 1:].mean(axis=1)

# Replace the 21 days values with their mean for each ID
for column in data.columns[1:]:
    data[column] = means
data = data.round().astype(int)
data.to_csv('mean_0826_5_5.csv',index=False)
data

<ipython-input-3-85cb60fd02bf>:55: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)


,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
15886,15886,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
15887,15887,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
15888,15888,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


# LSTM ENS

In [ ]:
import pandas as pd
import numpy as np
# Reload the provided files for ensemble
#DLinear35_df = pd.read_csv('/content/sixth_ensemble_result_DLinear.csv')
#lstm105_df = pd.read_csv('/content/bilstm_nofeature_105_imp3_noiseX_41.csv')
#lstm35_df = pd.read_csv('/content/bilstm_nofeature_35_imp3_noiseX_41 (1).csv')
#lstm70_df = pd.read_csv('/content/bilstm_nofeature_70_imp3_noiseX_41.csv')
#gru105_df = pd.read_csv('/content/gru_nofeature_105_imp3_noiseX_41.csv')
#gru35_df = pd.read_csv('/content/gru_nofeature_35_imp3_noiseX_41.csv')
#gru70_df = pd.read_csv('/content/gru_nofeature_70_imp3_noiseX_41.csv')
#Linear35_df = pd.read_csv('/content/fifth_ensemble_result_linear.csv')
#NLinear35_df = pd.read_csv('/content/NLinear_35_.csv')
lstm28 = pd.read_csv('/content/lstm_28_ens_exclude_topbottom.csv')
lstm35 = pd.read_csv('/content/lstm_35_ens_exclude_topbottom.csv')
lstm56 = pd.read_csv('/content/lstm_56_ens_exclude_topbottom.csv')
lstm70 = pd.read_csv('/content/lstm_70_ens_exclude_topbottom.csv')
lstm84 = pd.read_csv('/content/lstm_84_ens_exclude_topbottom.csv')


ensemble_6files_df = lstm28[['ID']].copy()

weights = {
    'lstm28' : 0.2,
    'lstm35' : 0.2,
    'lstm56' : 0.2,
    'lstm70' : 0.2,
    'lstm84' : 0.2,
}


for date_column in lstm28.columns[1:]:
    ensemble_6files_df[date_column] = (
        #lstm35_df[date_column] * weights['bilstm35'] +
        #lstm70_df[date_column] * weights['bilstm70'] +
        #lstm105_df[date_column] * weights['bilstm105'] +
        #gru35_df[date_column] * weights['gru35'] +
        #gru70_df[date_column] * weights['gru70'] +
        #gru105_df[date_column] * weights['gru105'] +
        #DLinear35_df[date_column] * weights['DLinear35'] +
        #Linear35_df[date_column] * weights['Linear35'] +
        lstm35[date_column] * weights['lstm35'] +
        lstm28[date_column] * weights['lstm28'] +
        lstm56[date_column] * weights['lstm56'] +
        lstm70[date_column] * weights['lstm70'] +
        lstm84[date_column] * weights['lstm84']
        #NLinear35_df[date_column] * weights['NLinear35']
    )


ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)
import pandas as pd
import numpy as np
train_data = pd.read_csv("/content/drive/MyDrive/Aimers3/train.csv")

# Filter dates from 2022-01-01 to 2023-04-04 for train ensemble_6files_df
date_cols = [col for col in train_data.columns if '2022-01-01' <= col <= '2023-04-04']
filtered_train_data = train_data[['ID'] + date_cols]

# Filter the IDs from bi3gr3lstf2_fl.csv where at least one of the 21 days values is 0
ids_at_least_one_zero_fl = ensemble_6files_df[ensemble_6files_df.iloc[:, 1:].apply(lambda x: (x == 0).any(), axis=1)]['ID']

# Calculate the median for the filtered IDs in train.csv, excluding 0 values
medians_fl_at_least_one_zero = filtered_train_data[filtered_train_data['ID'].isin(ids_at_least_one_zero_fl)].iloc[:, 1:].apply(lambda x: x[x != 0].median() if (x[x != 0].shape[0] > 0) else 0, axis=1)

# Replace the values in bi3gr3lstf2_fl.csv with the calculated medians
for idx in ids_at_least_one_zero_fl:
    ensemble_6files_df.loc[ensemble_6files_df['ID'] == idx, ensemble_6files_df.columns[1:]] = medians_fl_at_least_one_zero[idx]

ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].replace(0, 1)

means = ensemble_6files_df.iloc[:, 1:].mean(axis=1)

for column in ensemble_6files_df.columns[1:]:
    ensemble_6files_df[column] = means
ensemble_6files_df = ensemble_6files_df.round().astype(int)
data = ensemble_6files_df.copy()
data.iloc[:, 1:] = data.iloc[:, 1:].replace(0, 1)
means = data.iloc[:, 1:].mean(axis=1)

# Replace the 21 days values with their mean for each ID
for column in data.columns[1:]:
    data[column] = means
data = data.round().astype(int)
data.to_csv('lstm_ens_topbottom.csv',index=False)
data

<ipython-input-2-e9f3c56285b3>:50: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)


,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
1,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
15886,15886,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
15887,15887,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
15888,15888,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


# 그냥 앙상블

In [ ]:
import pandas as pd
import numpy as np
# Reload the provided files for ensemble
sublstf = pd.read_csv('/content/DLinear_jb.csv')
gru = pd.read_csv('/content/bilstm_nofeature_105_noimpute.csv')
bilstm_nofeature = pd.read_csv('/content/bilstm_nofeature_35_noimpute_41.csv')
DL_fillna = pd.read_csv('/content/bilstm_nofeature_70_noimpute.csv')
baseline_submit2 = pd.read_csv('/content/gru_nofeature_105_80p (1).csv')
bilstm_nofeature_90 = pd.read_csv('/content/gru_nofeature_35_80p.csv')
a = pd.read_csv('/content/gru_nofeature_70_80p.csv')
b = pd.read_csv('/content/lstf_jh.csv')
# Compute the ensemble using the six provided files
ensemble_6files_df = sublstf[['ID']].copy()

# For each date column, compute the mean across the six dataframes and round the result
for date_column in sublstf.columns[1:]:
    ensemble_6files_df[date_column] = (
        sublstf[date_column] +
        gru[date_column] +
        bilstm_nofeature[date_column] +
        DL_fillna[date_column] +
        baseline_submit2[date_column] +
        bilstm_nofeature_90[date_column] + a[date_column] + b[date_column]
    ) / 8

# Round the ensemble dataframe values
ensemble_6files_df = ensemble_6files_df.round().astype(int)

ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].replace(0, 1)

ensemble_6files_df.to_csv('bi3gr3lstf2_fl.csv',index=False)
ensemble_6files_df

In [ ]:
data = pd.read_csv('/content/fl_at_least_one_zero_modified_data.csv')
data.iloc[:, 1:] = data.iloc[:, 1:].replace(0, 1)
data

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,10.0,10.0,10.0,10.0,10.0,11.0,11.0,11.0,11.0,...,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
4,4,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
15886,15886,1.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,...,4.0,4.0,5.0,5.0,6.0,6.0,6.0,5.0,5.0,6.0
15887,15887,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
15888,15888,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# 0값 median 대체 앙상블

In [ ]:
import pandas as pd
import numpy as np
# Reload the provided files for ensemble
sublstf = pd.read_csv('/content/DLinear_0825.csv')
gru = pd.read_csv('/content/bilstm_nofeature_105_imp3_noiseX_41.csv')
bilstm_nofeature = pd.read_csv('/content/bilstm_nofeature_35_imp3_noiseX_41 (1).csv')
DL_fillna = pd.read_csv('/content/bilstm_nofeature_70_imp3_noiseX_41.csv')
baseline_submit2 = pd.read_csv('/content/gru_nofeature_105_imp3_noiseX_41.csv')
bilstm_nofeature_90 = pd.read_csv('/content/gru_nofeature_35_imp3_noiseX_41.csv')
a = pd.read_csv('/content/gru_nofeature_70_imp3_noiseX_41.csv')
b = pd.read_csv('/content/lstf_jh_777.csv')
# Compute the ensemble using the six provided files
ensemble_6files_df = sublstf[['ID']].copy()

# For each date column, compute the mean across the six dataframes and round the result
for date_column in sublstf.columns[1:]:
    ensemble_6files_df[date_column] = (
        sublstf[date_column] +
        gru[date_column] +
        bilstm_nofeature[date_column] +
        DL_fillna[date_column] +
        baseline_submit2[date_column] +
        bilstm_nofeature_90[date_column] + a[date_column] + b[date_column]
    ) / 8

# Round the ensemble dataframe values

ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)
ensemble_6files_df

In [ ]:
import pandas as pd
import numpy as np
# Reload the provided files for ensemble
DLinear35_df = pd.read_csv('/content/DLinear_0825.csv')
lstm105_df = pd.read_csv('/content/bilstm_nofeature_105_imp3_noiseX_41.csv')
lstm35_df = pd.read_csv('/content/bilstm_nofeature_35_imp3_noiseX_41 (1).csv')
lstm70_df = pd.read_csv('/content/bilstm_nofeature_70_imp3_noiseX_41.csv')
gru105_df = pd.read_csv('/content/gru_nofeature_105_imp3_noiseX_41.csv')
gru35_df = pd.read_csv('/content/gru_nofeature_35_imp3_noiseX_41.csv')
gru70_df = pd.read_csv('/content/gru_nofeature_70_imp3_noiseX_41.csv')
Linear35_df = pd.read_csv('/content/lstf_jh_777.csv')
# Compute the ensemble using the six provided files
ensemble_6files_df = sublstf[['ID']].copy()

# Define the weights for each model
weights = {
    'lstm35': 0.09,
    'lstm70': 0.08,
    'lstm105': 0.08,
    'gru35': 0.09,
    'gru70': 0.08,
    'gru105': 0.08,
    'DLinear35': 0.25,
    'Linear35': 0.25,
}


# Iterate over each date column (ignoring the first column, which is 'Date')
for date_column in DLinear35_df.columns[1:]:
    ensemble_6files_df[date_column] = (
        lstm35_df[date_column] * weights['lstm35'] +
        lstm70_df[date_column] * weights['lstm70'] +
        lstm105_df[date_column] * weights['lstm105'] +
        gru35_df[date_column] * weights['gru35'] +
        gru70_df[date_column] * weights['gru70'] +
        gru105_df[date_column] * weights['gru105'] +
        DLinear35_df[date_column] * weights['DLinear35'] +
        Linear35_df[date_column] * weights['Linear35']
    )


ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)
ensemble_6files_df

<ipython-input-12-f86024faebd6>:42: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].apply(np.floor).astype(int)


,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,1,2,2,2,2,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,1,1,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,2,1,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,2,2,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4,4,4,...,5,4,4,4,4,4,4,4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,7,9,11,12,12,12,11,11,12,...,13,13,12,12,12,12,13,13,12,12
15886,15886,2,4,5,6,6,6,6,6,6,...,5,5,5,6,6,6,5,5,5,6
15887,15887,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
15888,15888,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [ ]:
import pandas as pd
import numpy as np
train_data = pd.read_csv("/content/drive/MyDrive/Aimers3/train.csv")

# Filter dates from 2022-01-01 to 2023-04-04 for train ensemble_6files_df
date_cols = [col for col in train_data.columns if '2022-01-01' <= col <= '2023-04-04']
filtered_train_data = train_data[['ID'] + date_cols]

# Filter the IDs from bi3gr3lstf2_fl.csv where at least one of the 21 days values is 0
ids_at_least_one_zero_fl = ensemble_6files_df[ensemble_6files_df.iloc[:, 1:].apply(lambda x: (x == 0).any(), axis=1)]['ID']

# Calculate the median for the filtered IDs in train.csv, excluding 0 values
medians_fl_at_least_one_zero = filtered_train_data[filtered_train_data['ID'].isin(ids_at_least_one_zero_fl)].iloc[:, 1:].apply(lambda x: x[x != 0].median() if (x[x != 0].shape[0] > 0) else 0, axis=1)

# Replace the values in bi3gr3lstf2_fl.csv with the calculated medians
for idx in ids_at_least_one_zero_fl:
    ensemble_6files_df.loc[ensemble_6files_df['ID'] == idx, ensemble_6files_df.columns[1:]] = medians_fl_at_least_one_zero[idx]

ensemble_6files_df.iloc[:, 1:] = ensemble_6files_df.iloc[:, 1:].replace(0, 1)

means = ensemble_6files_df.iloc[:, 1:].mean(axis=1)

for column in ensemble_6files_df.columns[1:]:
    ensemble_6files_df[column] = means
ensemble_6files_df = ensemble_6files_df.round().astype(int)
#ensemble_6files_df.to_csv('check.csv',index=False)
ensemble_6files_df

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
15886,15886,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
15887,15887,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
15888,15888,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


# std

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Aimers3/train.csv")
sub = ensemble_6files_df.copy()

train_std = train.T.iloc[6:].apply(np.std) / train.T.iloc[6:].apply(np.std)[12336]
sub_std = sub.T.iloc[1:].apply(np.std) / sub.T.iloc[1:].apply(np.std)[12336]

for i in range(len(train_std)):
    if train_std[i] < sub_std[i]:
        sub.loc[i, '2023-04-05':] = train_std[i] / sub_std[i] * sub.loc[i, '2023-04-05':]

sub = sub.apply(np.round).astype(int)
#sub.to_csv('./bi3gr3lstf2_train_std.csv', index=False)
data = sub.copy()

# ens_mean

In [ ]:
# Calculate the mean of the 21 days for each ID
#data = pd.read_csv('/content/fl_at_least_one_zero_modified_data.csv')
data = ensemble_6files_df.copy()
data.iloc[:, 1:] = data.iloc[:, 1:].replace(0, 1)

#data = pd.read_csv('/content/fl_modified_data.csv')
means = data.iloc[:, 1:].mean(axis=1)

# Replace the 21 days values with their mean for each ID
for column in data.columns[1:]:
    data[column] = means
data = data.round().astype(int)
data.to_csv('mean_test_v3_25_25_25_25.csv',index=False)
data

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
15886,15886,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
15887,15887,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
15888,15888,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [ ]:
##v2
import pandas as pd

# Reload the provided CSV file into a DataFrame
data = pd.read_csv('/mnt/data/bi3gr3lstf2.csv')

# Define a function to compute the average after excluding top and bottom 7 values
def compute_average(row):
    sorted_values = sorted(row[1:])
    trimmed_values = sorted_values[7:-7]  # Exclude top and bottom 7 values
    return sum(trimmed_values) / len(trimmed_values)

# Calculate the trimmed mean for each ID
trimmed_means = data.apply(compute_average, axis=1)

# Replace the 21 days values with their trimmed mean for each ID
for column in data.columns[1:]:
    data[column] = trimmed_means
data = data.round()

data.head()


# pp_std

In [ ]:
def sort_ids_by_variance_corrected(predictions_df, high_variance_ids):
    # Calculate standard deviation for each ID across the 21 days
    std_dev = predictions_df.iloc[:, 1:].std(axis=1)

    # Filter the std_dev series to only have the high_variance_ids
    std_dev_filtered = std_dev[predictions_df['ID'].isin(high_variance_ids)]

    # Sort the IDs by their standard deviation in descending order
    sorted_ids = std_dev_filtered.sort_values(ascending=False).index.tolist()
    return sorted_ids

def sort_ids_by_variance(predictions_df, high_variance_ids):
    # Calculate standard deviation for each ID across the 21 days
    std_dev = predictions_df.iloc[:, 1:].std(axis=1)

    # Sort the IDs by their standard deviation in descending order
    sorted_ids = sorted(high_variance_ids, key=lambda x: std_dev[x], reverse=True)
    return sorted_ids

# Function to replace predictions for high variance IDs with the median sales and return replaced IDs
def replace_with_median_and_get_replaced_ids(predictions_df, median_sales, high_variance_ids):
    replaced_ids = []
    for id_ in high_variance_ids:
        if id_ in median_sales.index:
            predictions_df.loc[predictions_df['ID'] == id_, predictions_df.columns[1:]] = median_sales[id_]
            replaced_ids.append(id_)
    return predictions_df, replaced_ids
# Define the function to find high variance IDs

def find_high_variance_ids(predictions_df, threshold=250):
    # Calculate standard deviation for each ID across the 21 days
    std_dev = predictions_df.iloc[:, 1:].std(axis=1)
    return predictions_df[std_dev > threshold]['ID'].tolist()

def modified_complete_code():
    import pandas as pd
    import numpy as np

    # Load the prediction data and train data
    predictions_1 = pd.read_csv("/content/bi3gr3lstf2.csv")
    train_data = pd.read_csv("/content/drive/MyDrive/Aimers3/train.csv")

    # Get the high variance IDs
    high_variance_ids_1 = find_high_variance_ids(predictions_1)

    # Calculate the median sales for each ID excluding zeros
    sales_columns = train_data.columns[6:]
    non_zero_sales = train_data.copy()
    non_zero_sales[sales_columns] = non_zero_sales[sales_columns].replace(0, np.nan)
    median_sales = non_zero_sales[sales_columns].median(axis=1, skipna=True)

    # Replace predictions for high variance IDs and get the replaced IDs
    predictions_1_modified, replaced_ids_1 = replace_with_median_and_get_replaced_ids(predictions_1, median_sales, high_variance_ids_1)

    # Sort the high variance IDs using the corrected function
    sorted_high_variance_ids_1 = sort_ids_by_variance_corrected(predictions_1, high_variance_ids_1)

    # Save the modified predictions to CSV
    output_path_1 = "/content/bilstm_nofeature_150.csv"
    predictions_1_modified.to_csv(output_path_1, index=False)

    return output_path_1, replaced_ids_1, sorted_high_variance_ids_1

_, id, sort_id = modified_complete_code()
sort_id

In [ ]:
sort_id

[3481,
 3837,
 12355,
 12351,
 12347,
 12340,
 12336,
 12335,
 11595,
 8733,
 8708,
 8702,
 8531,
 8093,
 7148,
 7144,
 3948,
 3934,
 3849,
 14578]

In [ ]:
len(sort_id)

20

# pp_0to1

In [ ]:
# Load the prediction data again
predictions_2 = pd.read_csv("/content/bilstm_nofeature.csv")

# Replace all 0 values with 1 in the prediction columns only
predictions_2.iloc[:, 1:] = predictions_2.iloc[:, 1:].replace(0, 1)

# Save the modified predictions to CSV
output_path_2_corrected = "/content/bilstm_nofeature_0to1.csv"
predictions_2.to_csv(output_path_2_corrected, index=False)

output_path_2_corrected


'/content/bilstm_nofeature_0to1.csv'